# AST Analysis Notebook


In [18]:
# Import necessary libraries
import astor
import ast
print("Done importing")

Done importing


Show version 1 AST

In [19]:
compiled_ast = astor.parse_file("sample_files/funcs_v1.py")
print(astor.dump_tree(compiled_ast))

Module(
    body=[
        FunctionDef(name='assign_numbers',
            args=arguments(posonlyargs=[],
                args=[],
                vararg=None,
                kwonlyargs=[],
                kw_defaults=[],
                kwarg=None,
                defaults=[]),
            body=[
                Assign(targets=[Name(id='x')], value=Constant(value=1, kind=None), type_comment=None),
                Assign(targets=[Name(id='y')],
                    value=BinOp(left=Constant(value=2, kind=None), op=Add, right=Constant(value=3, kind=None)),
                    type_comment=None)],
            decorator_list=[],
            returns=None,
            type_comment=None)],
    type_ignores=[])


Show version 2 AST

In [20]:
compiled_ast = astor.parse_file("sample_files/funcs_v2.py")
print(astor.dump_tree(compiled_ast))

Module(
    body=[
        FunctionDef(name='assign_numbers',
            args=arguments(posonlyargs=[],
                args=[],
                vararg=None,
                kwonlyargs=[],
                kw_defaults=[],
                kwarg=None,
                defaults=[]),
            body=[
                Assign(targets=[Name(id='x')], value=Constant(value=1, kind=None), type_comment=None),
                Assign(targets=[Name(id='y')],
                    value=BinOp(left=Constant(value=2, kind=None), op=Add, right=Constant(value=3, kind=None)),
                    type_comment=None),
                Assign(targets=[Name(id='z')],
                    value=BinOp(
                        left=BinOp(left=Constant(value=4, kind=None), op=Add, right=Constant(value=5, kind=None)),
                        op=Add,
                        right=Constant(value=6, kind=None)),
                    type_comment=None)],
            decorator_list=[],
            returns=None,
            t

Create a class to print function names from an AST

In [21]:
class PrintFunction(ast.NodeTransformer):
    def visit_FunctionDef(self, node):
        self.generic_visit(node)
        print(node.name)
        for leaf in node.body:
            print(leaf)

PrintFunction().visit(compiled_ast)
print("Done.")

assign_numbers
Done.
